In [1]:
import os
class Config:
    # API Keys and credentials
    GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
    OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
    
    # Database configuration
    DB_HOST = os.getenv("DB_HOST", "localhost")
    DB_PORT = os.getenv("DB_PORT", "5432")
    DB_NAME = os.getenv("DB_NAME", "logistics_db")
    DB_USER = os.getenv("DB_USER", "user")
    DB_PASSWORD = os.getenv("DB_PASSWORD", "password")
    
    # API endpoints
    OPENWEATHER_BASE_URL = "https://api.openweathermap.org/data/2.5"
    GOOGLE_MAPS_BASE_URL = "https://maps.googleapis.com/maps/api"

    OPENWEATHER_API_KEY = "a3d0bcee74d1685d13e2e248aa3f8e59"

In [4]:
import requests

In [11]:
pip install osmnx pandas geopandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [117]:
import requests
import pandas as pd
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename=f'india_weather_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
)

class WeatherAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.session = self._create_session()
        self.request_count = 0
        self.last_request_time = time.time()
        
    def _create_session(self):
        session = requests.Session()
        retry_strategy = Retry(
            total=5,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)
        return session

    def handle_rate_limit(self):
        self.request_count += 1
        if time.time() - self.last_request_time >= 3600:
            self.request_count = 1
            self.last_request_time = time.time()
        
        if self.request_count >= 50:
            sleep_time = 60 - (time.time() - self.last_request_time)
            if sleep_time > 0:
                time.sleep(sleep_time)
            self.request_count = 0
            self.last_request_time = time.time()

def fetch_weather_for_coordinate(api, lat, lon):
    url = f'https://api.openweathermap.org/data/2.5/weather'
    params = {
        'lat': round(lat, 6),
        'lon': round(lon, 6),
        'appid': api.api_key,
        'units': 'metric'
    }
    
    api.handle_rate_limit()
    
    try:
        response = api.session.get(url, params=params, timeout=30)
        response.raise_for_status()
        weather_data = response.json()
        
        result = {
            'latitude': lat,
            'longitude': lon,
            'weather_main': weather_data.get('weather', [{}])[0].get('main'),
            'weather_description': weather_data.get('weather', [{}])[0].get('description'),
            'temperature': weather_data.get('main', {}).get('temp'),
            'feels_like': weather_data.get('main', {}).get('feels_like'),
            'temp_min': weather_data.get('main', {}).get('temp_min'),
            'temp_max': weather_data.get('main', {}).get('temp_max'),
            'pressure': weather_data.get('main', {}).get('pressure'),
            'humidity': weather_data.get('main', {}).get('humidity'),
            'wind_speed': weather_data.get('wind', {}).get('speed'),
            'wind_direction': weather_data.get('wind', {}).get('deg'),
            'cloudiness': weather_data.get('clouds', {}).get('all'),
            'country': weather_data.get('sys', {}).get('country'),
            'city_name': weather_data.get('name')
        }
        return result
        
    except Exception as e:
        logging.error(f"Error at {lat}, {lon}: {str(e)}")
        return None

def fetch_india_weather_data(api_key, batch_size=50, max_workers=3):
    # India's geographical boundaries
    india_latitudes = np.linspace(6.5, 37.5, 90)  # From south to north
    india_longitudes = np.linspace(68.0, 97.5, 180)  # From west to east
    
    weather_data_list = []
    api = WeatherAPI(api_key)
    
    # Create coordinate pairs for India
    coordinates = [(lat, lon) for lat in india_latitudes for lon in india_longitudes]
    total_points = len(coordinates)
    logging.info(f"Total coordinate points for India: {total_points}")
    
    # Process in batches
    for batch_num, i in enumerate(range(0, len(coordinates), batch_size)):
        batch = coordinates[i:i + batch_size]
        logging.info(f"Processing batch {batch_num + 1} of {len(coordinates)//batch_size + 1}")
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [
                executor.submit(fetch_weather_for_coordinate, api, lat, lon)
                for lat, lon in batch
            ]
            
            for future in as_completed(futures):
                result = future.result()
                if result and result.get('country') == 'IN':  # Only keep Indian locations
                    weather_data_list.append(result)
        
        # Save intermediate results
        # if weather_data_list:
        #     intermediate_df = pd.DataFrame(weather_data_list)
        #     intermediate_df.to_csv(f'india_weather_intermediate_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)
        
        # Progress update
        progress = (i + len(batch)) / total_points * 100
        print(f"Progress: {progress:.2f}% ({len(weather_data_list)} points collected)")
        
        # Small delay between batches
        time.sleep(1)
    
    # Create final dataframe and save
    final_df = pd.DataFrame(weather_data_list)
    final_df.to_csv(f'india_weather_final_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)
    return final_df

def main():
    WEATHER_API_KEY = "a3d0bcee74d1685d13e2e248aa3f8e59"
    
    logging.info("Starting India weather data collection")
    try:
        india_weather_df = fetch_india_weather_data(WEATHER_API_KEY)
        logging.info(f"Successfully completed data collection with {len(india_weather_df)} records")
        print("\nData Collection Summary:")
        print(f"Total locations collected: {len(india_weather_df)}")
        print("\nSample of collected data:")
        print(india_weather_df[['city_name', 'temperature', 'weather_main', 'humidity']].head())
    except Exception as e:
        logging.error(f"Fatal error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025-01-29 11:20:57,835 - INFO - Starting India weather data collection
2025-01-29 11:20:57,905 - INFO - Total coordinate points for India: 16200
2025-01-29 11:20:57,906 - INFO - Processing batch 1 of 325


Progress: 0.31% (0 points collected)


2025-01-29 11:21:58,903 - INFO - Processing batch 2 of 325


Progress: 0.62% (0 points collected)


2025-01-29 11:22:59,112 - INFO - Processing batch 3 of 325


...

Progress: 99.69% (5976 points collected)


2025-01-29 19:44:28,165 - INFO - Processing batch 324 of 325


Progress: 100.00% (5976 points collected)


2025-01-29 19:45:28,297 - INFO - Successfully completed data collection with 5976 records



Data Collection Summary:
Total locations collected: 5976

Sample of collected data:
         city_name  temperature weather_main  humidity
0  Manavālakurichi        26.29       Clouds        77
1  Manavālakurichi        26.31       Clouds        76
2        Kovakulam        26.33       Clouds        78
3    Kanniyākumāri        26.58       Clouds        77
4        Kovakulam        26.45       Clouds        78
